In [ ]:
!pip install sentence_transformers accelerate bitsandbytes chromadb evaluate rouge_score rank_bm25 nltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import files

# Загрузка файла через интерфейс Colab
uploaded = files.upload()

# После загрузки читаем файл (в примере используется текстовый файл)
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as file:
        document_text = file.read()

# Функция для разбиения текста на чанки
def split_into_chunks(text, chunk_size=200):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Разбиваем документ на чанки (по 200 слов)
chunks = split_into_chunks(document_text, chunk_size=200)
print(f"Количество чанков: {len(chunks)}")

Saving HR-бот (1).txt to HR-бот (1).txt
Количество чанков: 25


In [ ]:
import evaluate
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


rouge = evaluate.load("rouge")

# Загрузка метрики METEOR
meteor = evaluate.load("meteor")


# Токенизация документов для BM25
tokenized_chunks = [word_tokenize(chunk.lower()) for chunk in chunks]

# Инициализация BM25
bm25 = BM25Okapi(tokenized_chunks)

# Инициализация модели SentenceTransformer
embedding_model = SentenceTransformer("deepvk/USER-bge-m3")
# Генерируем эмбеддинги для чанков с использованием модели SentenceTransformer
chunk_embeddings = embedding_model.encode(chunks)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.34k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

In [ ]:
def get_prompt(user_query, context):
  return f"""Ты специалист тех поддержки, которые отвечает на вопросы пользователей по технической документации.
Тебе нужно отвечать на вопросы строго по документации, не придумывая ничего самому.
Отвечай на вопросы развернуто.
Не генерируй примеры кода, которые демонстрируют, как можно обрабатывать вопросы и ответы.
Отвечай на вопросы в деловом тоне.
Над ответом на вопрос думаю пошагово.
В ответах ни как не упоминай, что ты отвечаешь на вопрос по документации или какому-либо контексту.
В ответе не упоминай вопрос пользователя.
Не указывай никаких ссылок, если они не были заранее даны в документации.
Если ответа нет в документации, тогда пиши я не знаю.
content : отвечай на вопрос по этому контенксту: {context}.

user : вопрос пользователя звучит так: {user_query}"""

In [ ]:
generation_config = GenerationConfig(
    bos_token_id=128000,         # начальный токен, c которого начинается генерация
    eos_token_id=128009,         # токен остановки генерации
    pad_token_id=128000,         # id паддинга(одинаковая длина последовательностей)
    do_sample=True,             # Используем семплирование
    max_new_tokens=256,          # Ограничиваем количество новых токенов
    repetition_penalty=1.05,      # Штраф за повторения
    temperature=0.1,             # Степень случайности
    top_k=5,                    # Сужаем выбор токенов до k
    top_p=0.85                # Ограничиваем вероятность выбора
)
print(generation_config)

def hybrid_search(user_query, bm25_weight=0.5, embedding_weight=0.5, n_results=4):
    # Токенизация запроса для BM25
    tokenized_query = word_tokenize(user_query.lower())
    bm25_scores = bm25.get_scores(tokenized_query)

    # Получение эмбеддинга запроса
    query_embedding = embedding_model.encode([user_query])
    embedding_scores = cosine_similarity(query_embedding, chunk_embeddings)[0]

    # Комбинированные оценки
    combined_scores = bm25_weight * bm25_scores + embedding_weight * embedding_scores
    top_n_indices = np.argsort(combined_scores)[::-1][:n_results]

    # Возвращаем релевантные документы
    return [chunks[i] for i in top_n_indices]

def rag_with_chunks(query):
    # Ищем релевантные чанки
    retrieved_chunk = hybrid_search(query)

    # Создаем промпт с найденным контекстом
    prompt = get_prompt(query, retrieved_chunk)

    # Генерация ответа с помощью модели
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]  # Убираем токены промпта
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return output

references = """Наши ценности – трудолюбие, профессионализм и адекватность. Эти три качества мы особенно ценим в тебе и в других сотрудниках УЦСБ, ведь они помогают выстроить эффективную работу с Заказчиками и друг с другом.



Трудолюбие – это любовь к своему делу и стремление к результату.



Профессионализм – ответственность за результат своей работы и развитие своих компетенций.



Адекватность – умение маневрировать между требованиями Заказчика, возможностями Компании и трендами рынка. Способность сочетать настойчивость и доверие во взаимодействии с коллегами.



Миссия УЦСБ – безопасное использование современных технологий.



Миссия УЦСБ – формирование сильной инженерной школы на Урале. Поэтому прямо сейчас мы участвуем сразу в нескольких образовательных проектах:

1. Совместно с УрФУ разрабатываем передовую магистерскую программу по информационной безопасности.

2. Оснащаем учебные лаборатории Университета.

3. Проводим образовательные лекции и курсы для студентов УрФУ.



А еще вкладываемся в образование и развитие сотрудников. Подробнее об этом ты можешь узнать в пункте Возможности компании. """


# Пример запроса
query = "Каковы ценности и миссии компании? "
answer = rag_with_chunks(query)


rouge_result = rouge.compute(predictions=[answer], references=[references])
print("ROUGE:", rouge_result)

# Подсчет METEOR
meteor_result = meteor.compute(predictions=[answer], references=[references])
print("METEOR:", meteor_result)

print(f"Ответ: {answer}")


GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 256,
  "pad_token_id": 128000,
  "repetition_penalty": 1.05,
  "temperature": 0.1,
  "top_k": 5,
  "top_p": 0.85
}



/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


OutOfMemoryError: CUDA out of memory. Tried to allocate 810.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 663.06 MiB is free. Process 7942 has 14.10 GiB memory in use. Of the allocated memory 12.20 GiB is allocated by PyTorch, and 1.76 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)